In [2]:
import numpy as np
import matplotlib.pyplot as plt
import math


def right_triangle(grid_zero_level, T, M, N, lam):
    grid = np.zeros((M + 1, N + 1))
    grid[0, :] = grid_zero_level

    tau = T / M
    h = 1 / N
    coef = lam * tau / h

    if lam < 0 or coef > 1:
        print("Схема неустойчива")

    for m in range(1, M + 1):
        grid[m, :N] = (1 - coef) * grid[m - 1, :N] \
                        + coef * grid[m - 1, 1:N + 1]
        grid[m, N] = grid[m, 0]
    return grid


def cycle_marching(alpha, beta, gamma, F):
    N = F.shape[0] - 1
    C = np.zeros(N - 1)
    phi_q = np.zeros(N - 1)
    phi_p = np.zeros(N - 1)
    C[0] = -gamma / beta
    phi_p[0] = F[1] / beta
    phi_q[0] = -alpha / beta

    for i in range(1, N - 1):
        denominator = alpha * C[i - 1] + beta
        C[i] = -gamma / denominator
        phi_p[i] = (F[i + 1] - alpha * phi_p[i - 1]) / denominator
        phi_q[i] = -(alpha * phi_q[i - 1]) / denominator

    p = np.zeros(N - 1)
    q = np.zeros(N - 1)
    p[N - 2] = phi_p[N - 2]
    q[N - 2] = C[N - 2] + phi_q[N - 2]
    for i in reversed(range(N - 2)):
        p[i] = C[i] * p[i + 1] + phi_p[i]
        q[i] = C[i] * q[i + 1] + phi_q[i]

    Z_N = ((F[N] - gamma * p[0] - alpha * p[N-2]) /
           (beta + gamma * q[0] + alpha * q[N - 2]))

    appended = np.append(Z_N, p + q * Z_N)
    return np.append(appended, Z_N)


def upper_triangle(grid_zero_level, T, M, N, lam):
    grid = np.zeros((M + 1, N + 1))
    grid[0, :] = grid_zero_level
    tau = T / M
    h = 1 / N
    coef = lam * tau / h
    if tau / h > 1 / abs(lam):
        print("Циклическая прогонка неустойчива")

    alpha = coef / 2
    beta = 1
    gamma = -coef / 2
    for m in range(1, M + 1):
        grid[m, :] = cycle_marching(alpha, beta, gamma, grid[m - 1, :])

    return grid

In [3]:
N = 100
M = 40000
T = 1
tau = T / M
h = 1 / N
grid = np.zeros((M + 1, N + 1))
for n in range(N + 1):
    for m in range(M + 1):
        grid[m, n] = n * h + m * tau

precise_solution = np.sin(2 * 3.141592653589793238462643 * grid)
triangle_solution = upper_triangle(precise_solution[0], T, M, N, 1)
np.amax(np.abs(precise_solution - triangle_solution))

0.004161603540415962